# Von Digitalisaten zu Wissensgraphen: Eine automatisierte Extraktion und semantische Modellierung biographischer Daten am Beispiel von Lebensbeschreibungen der Herrnhuter Brüdergemeine

## 5.2 OCR-Erkennung

### OCR-Erkennung

In [31]:
import cv2
import os
import pytesseract

In [32]:
# Ordner mit den jpg-Dateien 
base_folder = "data/5.2_OCR-Erkennung/jpg"

# Ordner für die TXT-Dateien
output_base = "data/5.2_OCR-Erkennung/txt"

In [33]:
# Durchlaufe alle Unterordner
for folder_name in os.listdir(base_folder):
    folder_path = os.path.join(base_folder, folder_name)

    # Nur Verzeichnisse berücksichtigen
    if not os.path.isdir(folder_path):
        continue

    # Alle .jpg-Dateien holen
    jpeg_files = sorted([f for f in os.listdir(folder_path) if f.lower().endswith('.jpg')])

    if not jpeg_files:
        print(f"Keine .jpg-Dateien in Ordner: {folder_name}")
        continue

    # Ein neuer Ordner für jede Person erstellen (für txt-Output-Datei)
    output_folder = os.path.join(output_base, folder_name)
    os.makedirs(output_folder, exist_ok=True)

    # Bilder verarbeiten
    for jpeg_file in jpeg_files:
        img_path = os.path.join(folder_path, jpeg_file)
        img = cv2.imread(img_path)

        # Bildvorverarbeitung
        inverted_image = cv2.bitwise_not(img)
        gray_image = cv2.cvtColor(inverted_image, cv2.COLOR_BGR2GRAY)
        _, binary_image = cv2.threshold(gray_image, 120, 255, cv2.THRESH_BINARY)
        binary_image_contrast = cv2.convertScaleAbs(binary_image, alpha=2.0, beta=0)

        # OCR
        ocr_result = pytesseract.image_to_string(binary_image_contrast, lang="deu+frk")

        # TXT-Dateiname & Pfad
        txt_filename = os.path.splitext(jpeg_file)[0] + ".txt"
        txt_path = os.path.join(output_folder, txt_filename)

        # Speichern
        with open(txt_path, "w", encoding="utf-8") as txt_file:
            txt_file.write(ocr_result)

        print(f"Text gespeichert in: {txt_path}")


Text gespeichert in: data/5.2_OCR-Erkennung/txt/1851_Kölbing_Friedrich/28.txt
Text gespeichert in: data/5.2_OCR-Erkennung/txt/1851_Kölbing_Friedrich/29.txt
Text gespeichert in: data/5.2_OCR-Erkennung/txt/1851_Kölbing_Friedrich/30.txt
Text gespeichert in: data/5.2_OCR-Erkennung/txt/1851_Kölbing_Friedrich/31.txt
Text gespeichert in: data/5.2_OCR-Erkennung/txt/1851_Kölbing_Friedrich/32.txt
Text gespeichert in: data/5.2_OCR-Erkennung/txt/1851_Kölbing_Friedrich/33.txt
Text gespeichert in: data/5.2_OCR-Erkennung/txt/1851_Kölbing_Friedrich/34.txt
Text gespeichert in: data/5.2_OCR-Erkennung/txt/1851_Kölbing_Friedrich/35.txt
Text gespeichert in: data/5.2_OCR-Erkennung/txt/1851_Kölbing_Friedrich/36.txt
Text gespeichert in: data/5.2_OCR-Erkennung/txt/1851_Kölbing_Friedrich/37.txt
Text gespeichert in: data/5.2_OCR-Erkennung/txt/1851_Kölbing_Friedrich/38.txt
Text gespeichert in: data/5.2_OCR-Erkennung/txt/1851_Kölbing_Friedrich/39.txt
Text gespeichert in: data/5.2_OCR-Erkennung/txt/1851_Kölbing_Fri

### Verbesserung der OCR-Ergebnissen

In [34]:
import os
import re

In [35]:
# Textbereinigungsfunktion
def clean_text(text):
    if text is None:
        return ''
    
    # Punkt nach einer Zahl entfernen
    text = re.sub(r'(?<=\d)\.', '', text)
    
    # Silbentrennung am Zeilenende entfernen
    text = re.sub(r'-\n', '', text)

    # Exakte String-Ersetzungen (Formatierung/Zeilenumbrüche bleiben unangetastet)
    replacements = [
        ("ic)", "ich"),
        ("auc)", "auch"),
        ("nac)", "nach"),
        ("aud)", "auch"),
        ("Jh", "Ich"),
        ("IJ)", "Ich"),
        ("nad)", "nach"),
        ("fic", "sich"),
        ("i<ß", "ich"),
        ("ic?", "ich"),
        ("zurü>", "zurück"),
        ("durc<zumachen", "durchzumachen"),
        ("» ", ""),
        ('!', ''),
        ('/', ''),
        ("'", ''),
        (',', ''),
        ("„", ""),
        ("“", "")
    ]
    for old, new in replacements:
        text = text.replace(old, new)
        

    # KEINE Whitespace-Normalisierung, KEIN strip()
    return text

# Ordnerpfad
folder_path = r"data/5.2_OCR-Erkennung/txt"

# Alle .txt-Dateien im Ordner UND in Unterordnern durchgehen
for root, dirs, files in os.walk(folder_path):
    for filename in files:
        if filename.lower().endswith(".txt"):
            file_path = os.path.join(root, filename)

            # Zeilenenden beibehalten: newline="" verhindert Übersetzung der Zeilenumbrüche
            with open(file_path, "r", encoding="utf-8", newline="") as f:
                original = f.read()

            cleaned = clean_text(original)

            # Nur schreiben, wenn sich etwas geändert hat (schont Zeitstempel)
            if cleaned != original:
                with open(file_path, "w", encoding="utf-8", newline="") as f:
                    f.write(cleaned)
                # Optionales Feedback:
                # print(f"Bereinigt: {file_path}")


## 5.3 XML/TEI-Modellierung (Teil 1)

### Erstellung einer XML-Grundstruktur

In [36]:
import re
import os
import xml.etree.ElementTree as ET
from xml.dom import minidom

In [37]:
# Ordner mit txt-Dateien
base_folder = "data/5.2_OCR-Erkennung/txt"

In [39]:
# Die Dateien werden iteriert
for folder_name in os.listdir(base_folder):
    folder_path = os.path.join(base_folder, folder_name)

    if not os.path.isdir(folder_path):
        continue

    txt_files = sorted([f for f in os.listdir(folder_path) if f.lower().endswith('.txt')])

    if not txt_files:
        print(f"Keine .txt-Dateien in Ordner: {folder_name}")
        continue
    
    # === Erstelle die Root-Element der XML-Struktur ===
    root = ET.Element("TEI")
    root.set("version", "4.8.1")
    root.set("xmlns", "http://www.tei-c.org/ns/1.0")
    
        # === teiHeader-Struktur ===
    headerTEI_el = ET.SubElement(root, "teiHeader")
    fileDesc_el = ET.SubElement(headerTEI_el, "fileDesc")
    titleStmt_el = ET.SubElement(fileDesc_el, "titleStmt")
    
    titleStmt_el.append(ET.Comment("Titel muss definiert werden. Nach der Korrektur wird der Kommentar gelöscht"))
    ET.SubElement(titleStmt_el, "title")

    titleStmt_el.append(ET.Comment("Autor muss definiert werden. Nach der Korrektur wird der Kommentar gelöscht"))
    ET.SubElement(titleStmt_el, "author")

    # respStmt
    respStmt_el = ET.SubElement(titleStmt_el, "respStmt")
    resp_el = ET.SubElement(respStmt_el, "resp")
    resp_el.text = "XML-Modelling compiled by"
    name_el = ET.SubElement(respStmt_el, "name")
    name_el.text = "Svetlana Yakutina"

    # publicationStmt
    publicationStmt_el = ET.SubElement(fileDesc_el, "publicationStmt")
    publisher_el = ET.SubElement(publicationStmt_el, "publisher")
    orgName_el = ET.SubElement(publisher_el, "orgName")
    orgName_el.text = "Verlag der Unitäts-Buchhandlung"

    pubPlace_el = ET.SubElement(publicationStmt_el, "pubPlace")
    pubPlace_el.text = "Gnadau (Germany)"

    availability_el = ET.SubElement(publicationStmt_el, "availability")
    p_header_el = ET.SubElement(availability_el, "p")
    orgName_el_ = ET.SubElement(p_header_el, "orgName")
    orgName_el_.text = "Memorial University of Newfoundland"

    publicationStmt_el.append(ET.Comment("Date muss definiert werden. Nach der Korrektur wird der Kommentar gelöscht"))
    ET.SubElement(publicationStmt_el, "date")

    publicationStmt_el.append(ET.Comment("Ref muss definiert werden. Nach der Korrektur wird der Kommentar gelöscht"))
    ET.SubElement(publicationStmt_el, "ref")

    # sourceDesc Struktur
    sourceDesc_el = ET.SubElement(fileDesc_el, "sourceDesc")
    bibl_el = ET.SubElement(sourceDesc_el, "bibl", type="j")
    bibl_el.text = "Nachrichten aus der Brüder-Gemeine"

    # biblFull Struktur
    biblFull_el = ET.SubElement(sourceDesc_el, "biblFull")
    titleStmt_el_ = ET.SubElement(biblFull_el, "titleStmt")
    ET.SubElement(titleStmt_el_, "title").text = "Nachrichten aus der Brüder-Gemeine"
    ET.SubElement(titleStmt_el_, "author").text = "Unbekannt"

    editionStmt_el = ET.SubElement(biblFull_el, "editionStmt")
    ET.SubElement(editionStmt_el, "edition").text = "Digitale Ausgabe der Zeitschrift"

    publicationStmt_el_ = ET.SubElement(biblFull_el, "publicationStmt")
    ET.SubElement(publicationStmt_el_, "publisher").text = "Verlag der Unitäts-Buchhandlung"

    seriesStmt_el = ET.SubElement(biblFull_el, "seriesStmt")
    title_el__ = ET.SubElement(seriesStmt_el, "title", level="j", type="main")
    title_el__.text = "Nachrichten aus der Brüder-Gemeine"
    ET.SubElement(seriesStmt_el, "biblScope", unit="volume").text = "1856-1894"
    seriesStmt_el.append(ET.Comment("Erscheinungsjahr muss definiert werden"))
    seriesStmt_el.append(ET.Comment("Seiten müssen definiert werden"))
    ET.SubElement(seriesStmt_el, "biblScope", unit="issue")
    ET.SubElement(seriesStmt_el, "biblScope", unit="page")

    notesStmt_el = ET.SubElement(biblFull_el, "notesStmt")
    note_el = ET.SubElement(notesStmt_el, "note", type="fileFormat")
    note_el.text = "application/pdf"

    # msDesc Struktur
    msDesc_el = ET.SubElement(sourceDesc_el, "msDesc")
    msIdentifier_el = ET.SubElement(msDesc_el, "msIdentifier")
    ET.SubElement(msIdentifier_el, "repository").text = "Memorial University of Newfoundland"
    idno_el = ET.SubElement(msIdentifier_el, "idno")
    idno_el_ = ET.SubElement(idno_el, "idno", type="URLCatalogue")
    idno_el_.text = "https://dai.mun.ca/digital/nachrichten/"

    # === Textstruktur ===
    text_el = ET.SubElement(root, "text")
    body_el = ET.SubElement(text_el, "body")
    div_el = ET.SubElement(body_el, "div")

    # alle TXT-Dateien durchlaufen
    for txt_file in txt_files:
        txt_file_path = os.path.join(folder_path, txt_file)
        with open(txt_file_path, 'r', encoding='utf-8') as f:
            raw_text = f.read()

        #vor jeder nuen Seite wird geschloßenes pb-Tad zugefügt, das Attribut n bekommt den Dateinamen ohne txt
        file_base = os.path.splitext(txt_file)[0]
        pb_el = ET.SubElement(div_el, "pb", n=file_base)

        #alle Absätze wurden anhand von doppelten Zeilenumbruch in p-Tag gepackt
        paragraphs = raw_text.split('\n\n')
        for para in paragraphs:
            para = para.strip()
            if para:
                p_el = ET.SubElement(div_el, "p")
                p_el.text = para
                
    # XML-Datei erzeugen
    xml_str = ET.tostring(root, encoding='utf-8')
    pretty_xml = minidom.parseString(xml_str).toprettyxml(indent="  ")

    os.makedirs("data/5.3_TEI-Modellierung", exist_ok=True)
    output_filename = f"{folder_name}.xml"
    output_path = os.path.join("data/5.3_TEI-Modellierung", output_filename)

    with open(output_path, 'w', encoding='utf-8') as output_file:
        output_file.write(pretty_xml)

    print(f"XML-Datei erstellt: {output_path}")


XML-Datei erstellt: data/5.3_TEI-Modellierung/1851_Kölbing_Friedrich.xml
XML-Datei erstellt: data/5.3_TEI-Modellierung/1860_Suhl_D_W.xml
XML-Datei erstellt: data/5.3_TEI-Modellierung/1847_Schmitt_J_H.xml
XML-Datei erstellt: data/5.3_TEI-Modellierung/1856_Lemmerz_J.xml
XML-Datei erstellt: data/5.3_TEI-Modellierung/1855_Hoffmann_M_E.xml
XML-Datei erstellt: data/5.3_TEI-Modellierung/1893_Bonatz_J_A.xml
XML-Datei erstellt: data/5.3_TEI-Modellierung/1875_Breutel_J_C.xml
XML-Datei erstellt: data/5.3_TEI-Modellierung/1849_Hoffman_Johannes_Friedrich.xml
XML-Datei erstellt: data/5.3_TEI-Modellierung/1820_Schulz_Johann_Gottlieb.xml
XML-Datei erstellt: data/5.3_TEI-Modellierung/1884_Stuhl_S_E.xml
XML-Datei erstellt: data/5.3_TEI-Modellierung/1862_Lemmerz_A.xml
XML-Datei erstellt: data/5.3_TEI-Modellierung/1861_Kölbing_C_R.xml
XML-Datei erstellt: data/5.3_TEI-Modellierung/1862_Beck_J_C.xml
XML-Datei erstellt: data/5.3_TEI-Modellierung/1823_Bonatz_Johanna.xml
XML-Datei erstellt: data/5.3_TEI-Modell

## 5.4 Informationsextraktion aus Texten

### 5.4.1 Named Entity Recognition 

### 5.4.2 Relationsextraktion 

## 5.3 XML/TEI-Modellierung (Teil 2)

## 5.5 Datenabgleich

## 5.6 RDF

## 5.7 OWL

## 5.8 Turtle 